In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch_tensorrt

from torch.utils.tensorboard import SummaryWriter

import pytorch_quantization
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
from tqdm import tqdm

print(pytorch_quantization.__version__)

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.tools import get_cf_args, get_dummy_input, load_config
from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    quantize_tensorrt_transform_pass,
    test_quantize_tensorrt_transform_pass,
    fake_quantize_transform_pass,
    graph_calibration_pass,
    evaluate_fake_quantize_pass,
    fake_quantize_to_trt_pass
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model, get_tokenizer

set_logging_verbosity("info")


2.1.3


INFO     Set logging level to info
I0318 18:02:03.488311 140608133248832 logger.py:44] Set logging level to info


In [2]:
batch_size = 8
model_name = "vgg7"
dataset_name = "cifar10"

# batch_size = 1
# model_name = "facebook/opt-125m:patched"
# dataset_name = "cola"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/home/qizhu/Desktop/Work/mase/mase_output/test-accu-0.9332.ckpt"
# CHECKPOINT_PATH = "/home/qizhu/Desktop/Work/mase/mase_output/opt125.ckpt"

model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from /home/qizhu/Desktop/Work/mase/mase_output/test-accu-0.9332.ckpt
I0318 18:02:08.061976 140608133248832 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from /home/qizhu/Desktop/Work/mase/mase_output/test-accu-0.9332.ckpt


In [4]:
mg = MaseGraph(model=model)
ori_mg = MaseGraph(model=model)

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
# _ = model(**dummy_in)


mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-0.6623, -0.0972, -0.1828,  ..., -2.1179, -2.1179, -2.1179],
          [-0.8507,  0.0741, -0.4739,  ..., -2.1179, -2.1179, -2.1179],
          [-0.6109, -0.1828, -0.7479,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-0.5826,  0.0301,  0.0476,  ..., -2.0357, -2.0357, -2.0357],
          [-0.6702,  0.3102, -0.1800,  ..., -2.0357, -2.0357, -2.0357],
          [-0.3550,  0.1352, -0.3901,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, -

In [5]:
# pass_args = {
#     "by": "name",
#     "default": {"config": {"name": None}},
# }

# mg = fake_quantize_transform_pass(mg, pass_args)
# pass_args_eval = {
#     "data_module": data_module,
# }

# mg = evaluate_fake_quantize_pass(mg, pass_args_eval)

In [6]:
pass_args = {
    "by": "name",
    "default": {"config": {"name": None}},
    "feature_layers_0": {
        "config": {
            "FakeQuantize": True,
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "max",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "histogram",
                "quantize_axis": None,
            },
        }
    },    
    "classifier_0": {
        "config": {
            "FakeQuantize": True,
            "name": "fp16",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_1": {
        "config": {
            "FakeQuantize": True,
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_2": {
        "config": {
            "FakeQuantize": True,
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_3": {
        "config": {
            "FakeQuantize": True,
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
}

mg = fake_quantize_transform_pass(mg, pass_args)


AssertionError: 

In [7]:
pass_args_calibrate = {
    "calibrator": "percentile",
    "percentiles": [99],
    "data_module": data_module,
    "num_batches": 100,
}

In [6]:
# for node in mg.fx_graph.nodes:
#     print(node)
print(dir(mg.fx_graph))

['__class__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_codegen', '_graph_namespace', '_insert', '_len', '_owning_module', '_python_code', '_root', '_target_to_str', '_tracer_cls', '_tracer_extras', '_used_names', 'call_function', 'call_method', 'call_module', 'create_node', 'eliminate_dead_code', 'erase_node', 'get_attr', 'graph_copy', 'inserting_after', 'inserting_before', 'lint', 'node_copy', 'nodes', 'on_generate_code', 'output', 'owning_module', 'placeholder', 'print_tabular', 'process_inputs', 'process_outputs', 'python_code', 'set_codegen']


In [8]:
graph_calibration_pass(mg,  pass_args_calibrate)
pass_args_eval = {
    "data_module": data_module,
}

mg = evaluate_fake_quantize_pass(mg, pass_args_eval)

W0318 09:07:59.987377 140021005461312 tensor_quantizer.py:174] Disable MaxCalibrator
W0318 09:07:59.987811 140021005461312 tensor_quantizer.py:174] Disable HistogramCalibrator
W0318 09:07:59.988284 140021005461312 tensor_quantizer.py:174] Disable HistogramCalibrator
W0318 09:07:59.988708 140021005461312 tensor_quantizer.py:174] Disable MaxCalibrator
W0318 09:07:59.989145 140021005461312 tensor_quantizer.py:174] Disable HistogramCalibrator
W0318 09:07:59.989608 140021005461312 tensor_quantizer.py:174] Disable MaxCalibrator
W0318 09:07:59.990584 140021005461312 tensor_quantizer.py:238] Load calibrated amax, shape=torch.Size([]).
W0318 09:07:59.991120 140021005461312 tensor_quantizer.py:239] Call .cuda() if running on GPU after loading calibrated amax.
W0318 09:07:59.993585 140021005461312 tensor_quantizer.py:238] Load calibrated amax, shape=torch.Size([]).
W0318 09:07:59.994244 140021005461312 tensor_quantizer.py:238] Load calibrated amax, shape=torch.Size([]).
W0318 09:07:59.995028 1400

feature_layers.0._input_quantizer       : TensorQuantizer(8bit fake per-tensor amax=4.2222 calibrator=MaxCalibrator scale=1.0 quant)
feature_layers.0._weight_quantizer      : TensorQuantizer(8bit fake per-tensor amax=0.2797 calibrator=HistogramCalibrator scale=1.0 quant)
classifier.0._input_quantizer           : TensorQuantizer(8bit fake per-tensor amax=2.2956 calibrator=HistogramCalibrator scale=1.0 quant)
classifier.0._weight_quantizer          : TensorQuantizer(8bit fake per-tensor amax=0.4703 calibrator=MaxCalibrator scale=1.0 quant)
classifier.2._input_quantizer           : TensorQuantizer(8bit fake per-tensor amax=9.6107 calibrator=HistogramCalibrator scale=1.0 quant)
classifier.2._weight_quantizer          : TensorQuantizer(8bit fake per-tensor amax=0.3434 calibrator=MaxCalibrator scale=1.0 quant)
Succeeded calibrating model in pyTorch!
Average execute time for one batch: 1.17ms
Total accuracy: 85.70%


In [9]:
pass_args = {
    "onnxFile": "onnx_test.onnx",
    "engineFile": "engine_test.plan",
    "dataloader": data_module.test_dataloader,
}
mg = fake_quantize_to_trt_pass(mg, pass_args)

/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/pytorch_quantization-2.1.3-py3.10-linux-x86_64.egg/pytorch_quantization/tensor_quant.py:378: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if min_amax < 0:
/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/pytorch_quantization-2.1.3-py3.10-linux-x86_64.egg/pytorch_quantization/tensor_quant.py:381: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  max_bound = torch.tensor((2.0**(num_bits - 1 + int(unsigned))) - 1.0, device=amax.device)
/home/qizhu

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Succeeded finding ONNX file!
Succeeded parsing .onnx file!
Succeeded building engine!
[ 0]Input -> DataType.FLOAT (-1, 3, 32, 32) (8, 3, 32, 32) input
[ 1]Output-> DataType.FLOAT (-1, 10) (8, 10) output
Succeeded running model in TensorRT!
Average execute time for one batch: 0.09ms
Total accuracy: 86.32%


In [10]:
pass_args_eval = {
    "data_module": data_module,
}

mg = evaluate_fake_quantize_pass(mg, pass_args_eval)

Average execute time for one batch: 1.13ms
Total accuracy: 90.11%


In [11]:
for node in mg.fx_graph.nodes:
    print(node.name)

x
feature_layers_0
feature_layers_1
feature_layers_2
feature_layers_3
feature_layers_4
feature_layers_5
feature_layers_6
feature_layers_7
feature_layers_8
feature_layers_9
feature_layers_10
feature_layers_11
feature_layers_12
feature_layers_13
feature_layers_14
feature_layers_15
feature_layers_16
feature_layers_17
feature_layers_18
feature_layers_19
feature_layers_20
view
classifier_0
classifier_1
classifier_2
classifier_3
last_layer
output


# OPT-125M


In [2]:
wikitext_info = get_dataset_info("wikitext2")
opt = get_model(
    "facebook/opt-125m:patched",
    task="lm",
    dataset_info=wikitext_info,
    pretrained=True,
)
opt_tokenizer = get_tokenizer("facebook/opt-125m:patched")

print(f"prepare data")
# Get data module for dummy inputs
data_module = MaseDataModule(
    name="wikitext2",
    batch_size=2,
    num_workers=os.cpu_count(),
    max_token_len=128,
    tokenizer=opt_tokenizer,
    load_from_cache_file=True,
    model_name="facebook/opt-125m@patched",
)
data_module.prepare_data()
data_module.setup()

/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


prepare data


In [26]:
model_info = get_model_info("facebook/opt-125m:patched")
cf_args = get_cf_args(model_info=model_info, task="lm", model=opt)

mg = MaseGraph(model=opt, cf_args=cf_args)

# dummy_in = get_dummy_input(model_info, data_module=data_module, task="lm")

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="lm",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
if len(mg.model.additional_inputs) > 0:
    dummy_in = dummy_in | mg.model.additional_inputs

# Generate graph and initialize metadata
# print(f"init metadata")
mg, _ = init_metadata_analysis_pass(mg, pass_args=None)
# mg, _ = add_common_metadata_analysis_pass(mg, pass_args={"dummy_in": dummy_in})
# mg, _ = add_software_metadata_analysis_pass(mg, None)

# for node in mg.fx_graph.nodes:
#     print(node.meta['mase'].module)


In [30]:
# print(dummy_in)
mg.model.graph.print_tabular()
mg.model.recompile()

opcode         name                                           target                                                                      args                                                                                                  kwargs
-------------  ---------------------------------------------  --------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
placeholder    input_ids                                      input_ids                                                                   (None,)                                                                                               {}
placeholder    attention_mask                                 at

PythonCode(src='\ntorch.fx._symbolic_trace.wrap("chop_models_patched_opt_patched_utils_opt_patched_opt_patched_shape_assertion_4")\ntorch.fx._symbolic_trace.wrap("chop_models_patched_opt_patched_utils_opt_patched_opt_patched_fn_prepare_decoder_attention_mask")\ntorch.fx._symbolic_trace.wrap("chop_models_patched_opt_patched_utils_opt_patched_opt_patched_fn_calculate_causal_lm_loss")\n\ndef forward(self, input_ids : torch.LongTensor = None, attention_mask : typing_Union[torch.Tensor,NoneType] = None, head_mask_1 = None, past_key_values_1 = None, inputs_embeds_1 = None, labels : typing_Union[torch.LongTensor,NoneType] = None, use_cache_1 = None, output_attentions_1 = None, output_hidden_states_1 = None, return_dict_1 = None) -> typing_Union[typing_Tuple,transformers_modeling_outputs_CausalLMOutputWithPast]:\n    _assert_is_none = torch.fx._symbolic_trace._assert_is_none(head_mask_1, \'head_mask has been specialized to have value None but got another value\');  head_mask_1 = None\n    _ass

In [29]:
mg.model(**dummy_in)

{'loss': 0,
 'logits': tensor([[[ -4.1079,  -4.1031,   5.2360,  ...,  -4.1251,  -4.1582,  -4.2522],
          [ -7.0824,  -7.0917,   2.2125,  ...,  -6.9990,  -6.9708,  -6.8121],
          [ -6.8083,  -6.8242,   2.1220,  ...,  -6.8255,  -6.6913,  -7.0274],
          ...,
          [ -8.3876,  -8.3888,   0.4175,  ...,  -8.3798,  -8.4188,  -8.5476],
          [ -7.1151,  -7.1061,  -1.0842,  ...,  -7.1819,  -7.1834,  -7.0982],
          [ -8.3191,  -8.3353,   0.6130,  ...,  -8.3036,  -8.3008,  -8.3952]],
 
         [[ -6.9732,  -6.9660,   1.8757,  ...,  -6.9888,  -6.9423,  -7.0070],
          [ -9.4373,  -9.4374,   2.1275,  ...,  -9.5177,  -9.3029,  -9.7325],
          [ -9.5896,  -9.5981,  -1.2567,  ...,  -9.6837,  -9.4658,  -9.6430],
          ...,
          [ -9.0750,  -9.0574,   2.4102,  ...,  -9.0348,  -9.0470,  -9.0953],
          [ -9.0326,  -9.0244,   5.6410,  ...,  -9.0118,  -9.0628,  -9.1411],
          [-10.8525, -10.8510,   7.0259,  ..., -10.7477, -10.8022, -10.7488]]],
       

In [7]:
for node in mg.fx_graph.nodes:
    args, kwargs = None, None
    if node.op == "placeholder":
        result = dummy_in[node.name]
        print(result)

tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
None
None
None
tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
False
False
False
True


In [5]:
for node in mg.fx_graph.nodes:
    # if node.op == "call_module":
    print(node.op)

placeholder
placeholder
placeholder
call_function
placeholder
call_function
placeholder
call_function
placeholder
placeholder
call_function
call_function
placeholder
call_function
call_function
placeholder
call_function
call_function
placeholder
call_function
call_function
call_function
call_function
call_method
call_module
call_function
call_function
call_function
call_function
call_function
call_module
call_function
call_module
call_module
call_function
call_function
call_function
call_function
call_function
call_method
call_method
call_module
call_module
call_module
call_module
call_function
call_function
call_function
call_method
call_module
call_module
call_function
call_function
call_function
call_function
call_function
call_method
call_method
call_module
call_module
call_module
call_module
call_function
call_function
call_function
call_method
call_module
call_module
call_function
call_function
call_function
call_function
call_function
call_method
call_method
call_module
call_mod

In [25]:
import time
for i, batch in enumerate(data_module.val_dataloader()):
    # print(i, batch.keys())
    if len(mg.model.additional_inputs) > 0:
        batch = batch | mg.model.additional_inputs
    curr_time = time.time()
    outputs = mg.model(**batch)
    print(f"Batch {i} took {time.time() - curr_time} seconds")
    # print(outputs)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [11]:
for k, v in dummy_in.items():
    print(k, v)

input_ids tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
attention_mask tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
labels tensor(..., device='meta', size=(1, 128), dtype=torch.int64)


In [28]:
dummy_in = {
    k: v.cuda() if isinstance(v, torch.Tensor) else v
    for k, v in dummy_in.items()
}

In [18]:
pass_args = {
    "precision": 'int8',                                                     # collect weight statistics for linear layers
    "nCalibration": 10,                                                # collect activation statistics for relu layers
    "dummy_in": dummy_in,
    "onnxFile": 'model.onnx',
    "cacheFile": 'model.INT8Cache',  
    "engineFile": 'model.plan'
}
engine = quantize_tensorrt_transform_pass(mg, pass_args)

/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/torch/__init__.py:1209: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
/home/qizhu/Desktop/Work/mase/machop/chop/models/patched/opt_patched/utils_opt_patched.py:127: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert (
/home/qizhu/Desktop/Work/mase/machop/chop/models/patched/opt_patched/utils_opt_patched.py:67: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value 

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



RuntimeError: output 1 (0
[ CPULongType{} ]) of traced region did not have observable data dependence with trace inputs; this probably indicates your program cannot be understood by the tracer.